In [1]:
#include <memory>
#include <iostream>
#include <map>
#include <string>

* 1 Basic operation of std::unique_ptr
* 2 re-implenent std::unique_ptr
* 3 what happens when std::unique_ptr in a function declration
* 4 what happens when std::unique_ptr as member variable in a class

In [2]:
class Resource
{
public:
    int number;
    
    Resource(int n) {
        number = n;
        std::cout << "Resource acquired\n"; 
    }
    
    ~Resource() { 
        std::cout << "Resource destroyed\n"; 
    }
    
    friend std::ostream& operator<<(std::ostream& out, const Resource& res){
        out << "I am a resource\n";
        return out;
    }
};

# 1 Basic operation of std::unique_ptr

## 1.a How to construct a std::unique_ptr

method1: use new

In [3]:
std::unique_ptr<Resource> res1{new Resource(5)}

Resource acquired


method2: use std:make_unique

In [4]:
std::unique_ptr<Resource> res2 = std::make_unique<Resource>(5);

Resource acquired


In [5]:
{
    std::unique_ptr<Resource> res3 = std::make_unique<Resource>(5);
    
    //dereference the pointer
    std::cout << *res3 << std::endl;
    
    // access the object pointer points to
    std::cout << "the attribute of heap-object: " << res3->number << std::endl;
}

Resource acquired
I am a resource

the attribute of heap-object: 5
Resource destroyed


**Because the std::unique_ptr is allocated on the stack here(a local variable), it’s guaranteed to eventually go out of scope, and when it does, it will delete the Resource it is managing.**

## 1.b how std::unique_ptr work and the lifetime of smart_pointer and  the Resource it point to.

In [6]:
int auto_release_heap_memory_when_unique_ptr_go_out_of_scope(){
    
    std::unique_ptr<Resource> resource_ptr =  std::make_unique<Resource>(2);
    
    std::cout << "unique_ptr is going out of scope " << std::endl;
    return 0;
}; //res1 go out of scope, and then release Resource

In [7]:
int a = auto_release_heap_memory_when_unique_ptr_go_out_of_scope();

Resource acquired
unique_ptr is going out of scope 
Resource destroyed


std::unique_ptr is a smart pointer that owns and manages another object through a pointer and disposes of that object when the unique_ptr goes out of scope.

The object is disposed of, using the associated deleter when either of the following happens:

* the managing unique_ptr object is destroyed
* the managing unique_ptr object is assigned another pointer via operator= or reset().

In [8]:
int control_the_lifetime_of_heap_object_by_unique_ptr_member_function(){
    
    std::unique_ptr<Resource> resource_ptr = std::make_unique<Resource>(2);
    std::cout << "access the attribute of heap-object: " << resource_ptr->number << std::endl;
    std::cout << "release the heap-memory-object by mannual" << std::endl;
    resource_ptr.reset(); // is equally to resource_ptr = nullptr,// delete the object, leaving unique_ptr empty
    std::cout << "unique_ptr is going out of scope " << std::endl;
    return 0;
}; //res1 go out of scope, and then release Resource

In [9]:
int b = control_the_lifetime_of_heap_object_by_unique_ptr_member_function();

Resource acquired
access the attribute of heap-object: 2
release the heap-memory-object by mannual
Resource destroyed
unique_ptr is going out of scope 


## 1.c std::unique_ptr: copy construct and copy assgnment disabled and you can only it's move semantic

copy construction and copy assignment are disabled

That is, std::unique_ptr should completely own the object it manages, not share that ownership with other classes.

![](./images/unique_ptr.png)

In [10]:
int test_rule_of_five(){
    std::unique_ptr<Resource> p1{ new Resource{22} }; // Resource created here
	std::unique_ptr<Resource> p2{}; // Start as nullptr
 
	std::cout << "p1 is " << (static_cast<bool>(p1) ? "not null\n" : "null\n");
	std::cout << "p2 is " << (static_cast<bool>(p2) ? "not null\n" : "null\n");
 
    // //Copy construction is not allowed
    //unique_ptr<MyObject> p3(p1);
	
    // res2 = res1; // Won't compile: copy assignment is disabled
    
    // the output of std:move is a rvalue, the call the move assignment operator rather than copy assignment which is disabled 
	p2 = std::move(p1); // res2 assumes ownership, res1 is set to null
 
	std::cout << "Ownership transferred\n";
 
	std::cout << "p1 is " << (static_cast<bool>(p1) ? "not null\n" : "null\n");
	std::cout << "p2 is " << (static_cast<bool>(p2) ? "not null\n" : "null\n");
     
    std::cout << *p2 << std::endl;
    
	return 0;
};

In [11]:
int b = test_rule_of_five();

Resource acquired
p1 is not null
p2 is null
Ownership transferred
p1 is null
p2 is not null
I am a resource

Resource destroyed


if you want to transfer the contents managed by std::unique_ptr, you must use move semantics. In the program above, we accomplish this via std::move (which converts res1 into an r-value, which triggers a move assignment instead of a copy assignment).

# 2 reimplement a unique_ptr

In [12]:
template <typename T>
class UniquePtr {
    private:
    T* ptr_;
    
    public:
    
    UniquePtr(T* ptr) : ptr_{ptr} {}

    UniquePtr() : ptr_{nullptr} {}

    UniquePtr(const UniquePtr& p) = delete;
    UniquePtr& operator=(const UniquePtr& p) = delete;

    UniquePtr(UniquePtr&& p) {
        this->ptr_ = p.ptr_;
        p.ptr_ = nullptr;
    }

    UniquePtr& operator=(UniquePtr&& p) {
        clean();
        this->ptr_ = p.ptr_;
        p.ptr_ = nullptr;
        return *this;
    }

    T* get() const { return ptr_; }

    T* operator->() const { return ptr_; }

    T& operator*() const { return *ptr_; }

    operator bool() const { return ptr_; }

    ~UniquePtr() { clean(); }

   private:
    void clean() {
        if (ptr_) delete ptr_;
    }

};

In [13]:
{
    Resource* resource_ptr = new Resource(11);
    UniquePtr<Resource> a_custom_unique_ptr(resource_ptr);
    std::cout << "access heap-object attribute by unique_ptr: " << a_custom_unique_ptr->number << std::endl;
    UniquePtr<Resource> another_custom_unique_ptr(std::move(a_custom_unique_ptr));
    std::cout << "another_custom_unique_ptr is going out of scope " << std::endl;
} // 

Resource acquired
access heap-object attribute by unique_ptr: 11
another_custom_unique_ptr is going out of scope 
Resource destroyed


# 3 what happens when std::unique_ptr in a function declration

## 3-1: Passing std::unique_ptr to a function

If you want the function to take ownership of the contents of the pointer, pass the std::unique_ptr by value. Note that because copy semantics have been disabled, you’ll need to use std::move to actually pass the variable in.

Moving a unique_ptr vs. using get()?

In [14]:
void takeOwnership(std::unique_ptr<Resource> res){
     if (res)
          std::cout << *res << '\n';
}; // the Resource is destroyed here


std::unique_ptr has an overloaded operator* and operator-> that can be used to return the resource being managed. Operator* returns a reference to the managed resource, and operator-> returns a pointer.

Remember that std::unique_ptr may not always be managing an object -- either because it was created empty (using the default constructor or passing in a nullptr as the parameter), or because the resource it was managing got moved to another std::unique_ptr. So before we use either of these operators, we should check whether the std::unique_ptr actually has a resource. Fortunately, this is easy: std::unique_ptr has a cast to bool that returns true if the std::unique_ptr is managing a resource.

In [15]:
int test_unique_pointer_as_function_parameter1(){
    
    auto ptr{ std::make_unique<Resource>(111)};
 
//    takeOwnership(ptr); // This doesn't work, need to use move semantics
    takeOwnership(std::move(ptr)); // ok: use move semantics
 
    std::cout << "Ending program\n";
 
    return 0;
};

int d1 = test_unique_pointer_as_function_parameter1();

Resource acquired
I am a resource

Resource destroyed
Ending program


However, most of the time, you won’t want the function to take ownership of the resource. Although you can pass a std::unique_ptr by reference (which will allow the function to use the object without assuming ownership), you should only do so when the called function might alter or change the object being managed.

Instead, it’s better to just pass the resource itself (by pointer or reference, depending on whether null is a valid argument). This allows the function to remain agnostic of how the caller is managing its resources. To get a raw resource pointer from a std::unique_ptr, you can use the get() member function:

In [16]:
// The function only uses the resource, so we'll accept a pointer to the resource, not a reference to the whole std::unique_ptr<Resource>
void justUseResource(Resource* res){
    if (res)
          std::cout << *res << '\n';
};

In [17]:
int test_unique_pointer_as_function_parameter2(){
    
    auto ptr{ std::make_unique<Resource>(111)};
 
    justUseResource(ptr.get()); // ok
 
    std::cout << "Ending program\n";
 
    return 0;
};

int d2 = test_unique_pointer_as_function_parameter2();

Resource acquired
I am a resource

Ending program
Resource destroyed


## 3-2: Returning std::unique_ptr from a function

**return by value, raw pointer to Resource, unique_ptr reference??**

### case1: return by value: be carefully

In [18]:
std::unique_ptr<Resource> createUniquePointer(){
    // method 1:
    /*create a local unique pointer
    std::unique_ptr<Resource> p1(new Resource(111));
    
    Just wanted to add one more point that returning by value should be the default choice here because a named value in the return statement in the worst case, i.e. without elisions in C++11, C++14 and C++17 is treated as an rvalue. So for example the following function compiles with the -fno-elide-constructors flag
    return p1; // p1 will surrender ownership
    */
    
    //method2
    //return std:move(p1);
    
    //method3
    return std::make_unique<Resource>(111); //call the implicit std:move senmantics 
    
};


In [19]:
void test_unique_pointer_as_function_return(){
    std::unique_ptr<Resource> a1; //unique_ptr points to nothing
    a1=createUniquePointer(); //now a1 owns the object, here call the move assignment operator
    
    std::cout << *a1 << std::endl;
    std::cout << "program end " << std::endl;
};

test_unique_pointer_as_function_return();

Resource acquired
I am a resource

program end 
Resource destroyed


In the above code, createResource() returns a std::unique_ptr by value. If this value is not assigned to anything, the temporary return value will go out of scope and the Resource will be cleaned up. If it is assigned (as shown in main()), in C++14 or earlier, move semantics will be employed to transfer the Resource from the return value to the object assigned to (in the above example, ptr), and in C++17 or newer, the return will be elided. This makes returning a resource by std::unique_ptr much safer than returning raw pointers!

In general, you should not return std::unique_ptr by pointer (ever) or reference (unless you have a specific compelling reason to).

### case2: return by smart pointer reference: bad idea,

 it is indicative of bad design. Having a non-const reference means that you can steal the resource or replace it without having to offer separate methods.
 
 https://stackoverflow.com/questions/48863969/can-i-pass-a-unique-ptrs-reference-to-a-function

In [20]:
class TmContainer {
public:
    TmContainer() {
        // Create some sort of complex object on heap and store unique_ptr to it
        m_time = std::unique_ptr<tm>(new tm());
        // Store something meaningful in its fields
        m_time->tm_year = 42;
    }

    std::unique_ptr<tm>& time() { return m_time; }

private:
    std::unique_ptr<tm> m_time;
};

~~~
// Just create a handle to the managed object
auto& tm_ptr = tm_container.time();
do_something_to_tm(*tm_ptr);

// Steal the resource
std::unique_ptr<TmContainer> other_tm_ptr = std::move(tm_ptr);

// Replace the managed object with another one
tm_ptr = std::make_unique<TmContainer>;
~~~

### case3: return raw pointer of Resource

You should only return and accept smart pointers when you care about their ownership semantics. If you only care about what they're pointing to, you should instead return a reference or a raw pointer.

https://stackoverflow.com/questions/43400850/return-a-unique-ptr-by-reference

So I need the users of this class to get access to the order unique_ptr if found. However I'm not going to move the object out of the vector so I'm returning the unique_ptr as const ref. My implementation of the find_order method:

~~~
class order_collection {
    typedef std::unique_ptr<order> ord_ptr;
    typedef std::vector<ord_ptr> ord_ptr_vec;
    ord_ptr_vec orders;
    ord_ptr null_unique;
public:
    ...
    const ord_ptr & find_order(std::string);
    ....
~~~
        
~~~
const order_collection::ord_ptr & order_collection::find_order(std::string id) {
    auto it = std::find_if(orders.begin(),orders.end(),
                           [&](const order_collection::ord_ptr & sptr) {
                               return sptr->getId() == id;
                           });

    if (it == orders.end())
        return null_unique; // can't return nullptr here
    return *it;
}
~~~

Since I'm returning by reference I can't return a nullptr. If I try to do so, I get warning : returning reference to a temporary. And if nothing is found the program crashes. So I added a unique_ptr<order> member variable called null_unique and I return it when find doesn't find an order. This solves the problem and warning is gone and doesn't crash when no order is found.

However I'm doubting my solution as it make my class ugly. Is this the best practice for handling this situation?

Since you're returning a dummy null_unique, it is clear that the caller of the method doesn't care about the ownership semantics. You can also have a null state: you should therefore return a raw pointer:

~~~
order* order_collection::find_order(std::string id) {
    auto it = std::find_if(orders.begin(),orders.end(),
                           [&](const order_collection::ord_ptr & sptr) {
                               return sptr->getId() == id;
                           });

    if (it == orders.end())
        return nullptr;
    return it->get();
}
~~~

It doesn't really make sense to return a unique_ptr here, reference or otherwise. A unique_ptr implies ownership over the object, and those aren't really the semantics being conveyed by this code.

As suggested in the comments, simply returning a raw pointer is fine here, provided that your Project Design explicitly prohibits you or anyone on your team from calling delete or delete[] outside the context of the destructor of a Resource-owning object.

> +1 for the raw pointer solution. I will also add that it's very common to return iterators to collections for find-like operations. You'd return the end iterator on not found. Perhaps with C++17 people will start moving towards a std::optional approach, though

## 1-f: unique_pointer in dict

In [21]:
using ResourceMap = std::map<std::string, std::unique_ptr<Resource>>;

In [22]:
ResourceMap myMap;
myMap["000001"] = std::make_unique<Resource>(11);
myMap["000002"] = std::make_unique<Resource>(21);
myMap["000003"] = std::make_unique<Resource>(31);

Resource acquired
Resource acquired
Resource acquired


In [23]:
using RMI = std::map<std::string, std::unique_ptr<Resource>>::iterator;

In [24]:
RMI it = myMap.find("000002");

In [25]:
std::cout << it -> second -> number << std::endl;

21


In [26]:
{
    //std::unique_ptr<Resource> a_copy = std::move(it -> second);
    std::cout << it -> second -> number << std::endl;
}

21


In [27]:
for(auto i = myMap.begin(); i!=myMap.end(); i++){
    std::cout <<i->first << ":" << i->second -> number << std::endl;
}

000001:11
000002:21
000003:31


In [28]:
{
    std::unique_ptr<Resource> a_copy = std::move(it -> second);
    std::cout << a_copy -> number << std::endl;
}

21
Resource destroyed


In [29]:
//for(auto i = myMap.begin(); i!=myMap.end(); i++){
//    std::cout <<i->first << ":" << i->second -> number << std::endl;
//}

why ? because the value of the 000002 is nullptr because std::move

# 4 std::unique_ptr serves as a member variable in a class

You can, of course, use std::unique_ptr as a composition member of your class. This way, you don’t have to worry about ensuring your class destructor deletes the dynamic memory, as the std::unique_ptr will be automatically destroyed when the class object is destroyed.

However, if the class object is not destroyed properly (e.g. it is dynamically allocated and not deallocated properly), then the std::unique_ptr member will not be destroyed either, and the object being managed by the std::unique_ptr will not be deallocated.

>　https://stackoverflow.com/questions/15648844/using-smart-pointers-for-class-members

### context
I'm having trouble understanding the usage of smart pointers as class members in C++11. I have read a lot about smart pointers and I think I do understand how unique_ptr and shared_ptr/weak_ptr work in general. What I don't understand is the real usage. It seems like everybody recommends using unique_ptr as the way to go almost all the time. But how would I implement something like this:
~~~
class Device {
};

class Settings {
    Device *device;
public:
    Settings(Device *device) {
        this->device = device;
    }

    Device *getDevice() {
        return device;
    }
};    

int main() {
    Device *device = new Device();
    Settings settings(device);
    // ...
    Device *myDevice = settings.getDevice();
    // do something with myDevice...
}
~~~

Let's say I would like to replace the pointers with smart pointers. A unique_ptr would not work because of getDevice(), right? So that's the time when I use shared_ptr and weak_ptr? No way of using unique_ptr? Seems to me like for most cases shared_ptr makes more sense unless I'm using a pointer in a really small scope?
~~~
class Device {
};

class Settings {
    std::shared_ptr<Device> device;
public:
    Settings(std::shared_ptr<Device> device) {
        this->device = device;
    }

    std::weak_ptr<Device> getDevice() {
        return device;
    }
};

int main() {
    std::shared_ptr<Device> device(new Device());
    Settings settings(device);
    // ...
    std::weak_ptr<Device> myDevice = settings.getDevice();
    // do something with myDevice...
}
~~~

### answer

What is important here is to determine the appropriate ownership policy for your Device object, i.e. who is going to be the owner of the object pointed to by your (smart) pointer.

Is it going to be the instance of the Settings object alone? Will the Device object have to be destroyed automatically when the Settings object gets destroyed, or should it outlive that object?

In the first case, std::unique_ptr is what you need, since it makes Settings the only (unique) owner of the pointed object, and the only object which is responsible for its destruction.

Under this assumption, getDevice() should return a simple observing pointer (observing pointers are pointers which do not keep the pointed object alive). The simplest kind of observing pointer is a raw pointer:

~~~
#include <memory>

class Device {
};

class Settings {
    std::unique_ptr<Device> device;
public:
    Settings(std::unique_ptr<Device> d) {
        device = std::move(d);
    }

    Device* getDevice() {
        return device.get();
    }
};

int main() {
    std::unique_ptr<Device> device(new Device());
    Settings settings(std::move(device));
    // ...
    Device *myDevice = settings.getDevice();
    // do something with myDevice...
}

~~~

[NOTE 1: You may be wondering why I am using raw pointers here, when everybody keeps telling that raw pointers are bad, unsafe, and dangerous. Actually, that is a precious warning, but it is important to put it in the correct context: raw pointers are bad when used for performing manual memory management, i.e. allocating and deallocating objects through new and delete. When used purely as a means to achieve reference semantics and pass around non-owning, observing pointers, there is nothing intrinsically dangerous in raw pointers, except maybe for the fact that one should take care not to dereference a dangling pointer. - END NOTE 1]

[NOTE 2: As it emerged in the comments, in this particular case where the ownership is unique and the owned object is always guaranteed to be present (i.e. the internal data member device is never going to be nullptr), function getDevice() could (and maybe should) return a reference rather than a pointer. While this is true, I decided to return a raw pointer here because I meant this to be a short answer that one could generalize to the case where device could be nullptr, and to show that raw pointers are OK as long as one does not use them for manual memory management. - END NOTE 2]

The situation is radically different, of course, if your Settings object should not have the exclusive ownership of the device. This could be the case, for instance, if the destruction of the Settings object should not imply the destruction of the pointed Device object as well.

This is something that only you as a designer of your program can tell; from the example you provide, it is hard for me to tell whether this is the case or not.

To help you figure it out, you may ask yourself whether there are any other objects apart from Settings that are entitled to keep the Device object alive as long as they hold a pointer to it, instead of being just passive observers. If that is indeed the case, then you need a shared ownership policy, which is what std::shared_ptr offers:
~~~
#include <memory>

class Device {
};

class Settings {
    std::shared_ptr<Device> device;
public:
    Settings(std::shared_ptr<Device> const& d) {
        device = d;
    }

    std::shared_ptr<Device> getDevice() {
        return device;
    }
};

int main() {
    std::shared_ptr<Device> device = std::make_shared<Device>();
    Settings settings(device);
    // ...
    std::shared_ptr<Device> myDevice = settings.getDevice();
    // do something with myDevice...
}
~~~
Notice, that weak_ptr is an observing pointer, not an owning pointer - in other words, it does not keep the pointed object alive if all other owning pointers to the pointed object go out of scope.

The advantage of weak_ptr over a regular raw pointer is that you can safely tell whether weak_ptr is dangling or not (i.e. whether it is pointing to a valid object, or if the object originally pointed to has been destroyed). This can be done by calling the expired() member function on the weak_ptr object.

# 5 forbidden behavior in std::unique_ptr

There are two easy ways to misuse std::unique_ptrs, both of which are easily avoided. First, don’t let multiple classes manage the same resource.

## 5.1 don’t let multiple classes manage the same resource

~~~
Resource* res{ new Resource() };
std::unique_ptr<Resource> res1{ res };
std::unique_ptr<Resource> res2{ res };
~~~

While this is legal syntactically, the end result will be that both res1 and res2 will try to delete the Resource(double delete), which will lead to undefined behavior.

## 5.2 don’t manually delete the resource out from underneath the std::unique_ptr.

~~~
Resource* res{ new Resource() };
std::unique_ptr<Resource> res1{ res };
delete res;
~~~

In [30]:
{
    std::unique_ptr<Resource> up(new Resource{111});
    std::cout << up->number << std::endl;

    Resource* p = up.get();
    //delete p;
}

Resource acquired
111
Resource destroyed


# appendix 1: how to get the raw pointer from unique_ptr

**std::unique_ptr<T,Deleter>::get**

Returns a pointer to the managed object or nullptr if no object is owned.

In [31]:
std::unique_ptr<Resource> up(new Resource{111});
std::cout << up->number << std::endl;

Resource* p = up.get();
std::cout << *p << std::endl;
std::cout << p -> number << std::endl;

p -> number = 22;
std::cout << *p << std::endl;
std::cout << p -> number << std::endl;

std::cout << up->number << std::endl;

Resource acquired
111
I am a resource

111
I am a resource

22
22


***see***: the research subject of smart pointer is the pointer itself, which mean we can create a pointer which has many attributes we desired. such as the unique_ptr can not be copied constructed and copy assigned.

> std::unique_ptr is a smart pointer that owns and manages another object through a pointer and disposes of that object when the unique_ptr goes out of scope.

The object is disposed of, using the associated deleter when either of the following happens:

the managing unique_ptr object is destroyed

the managing unique_ptr object is assigned another pointer via operator= or reset()